In [1]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import math
import time
start_time = time.time()

In [3]:
players_df = pd.read_csv('players.csv')

week1_df = pd.read_csv('tracking_week_1.csv')
week2_df = pd.read_csv('tracking_week_2.csv')
week3_df = pd.read_csv('tracking_week_3.csv')
week4_df = pd.read_csv('tracking_week_4.csv')
week5_df = pd.read_csv('tracking_week_5.csv')
week6_df = pd.read_csv('tracking_week_6.csv')
week7_df = pd.read_csv('tracking_week_7.csv')
week8_df = pd.read_csv('tracking_week_8.csv')
week9_df = pd.read_csv('tracking_week_9.csv')

In [4]:
dataframes = [week1_df, week2_df, week3_df, week4_df, week5_df, week6_df, week7_df, week8_df, week9_df]

for df in dataframes:
    df['unique_key'] = df['gameId'].astype(str) + '_' + df['playId'].astype(str)

# Concatenate the filtered dataframes into a single dataframe
full_games_df = pd.concat(dataframes, ignore_index=True)

week1_df = pd.DataFrame()
week2_df = pd.DataFrame()
week3_df = pd.DataFrame()
week4_df = pd.DataFrame()
week5_df = pd.DataFrame()
week6_df = pd.DataFrame()
week7_df = pd.DataFrame()
week8_df = pd.DataFrame()
week9_df = pd.DataFrame()

In [5]:
# Returns the max value in quartile 3. Used to calculate "max" acceleration without including outliers.
def calcQ3Max(dataset):
    # Calculate the deltas between consecutive rows for each group of unique_keys
    deltas = dataset.groupby('unique_key')['s'].diff().dropna()
    
    q1 = np.percentile(deltas, 25) # Calculate the first quartile (Q1)
    
    q3 = np.percentile(deltas, 75) # Calculate the third quartile (Q3)
    
    iqr = q3 - q1 # Calculate the interquartile range (IQR)
    
    upper_bound = q3 + 1.5 * iqr # Define the upper bound for outliers
    
    q3_data = deltas[deltas <= upper_bound] # Filter values within the Q3 range
    
    max_in_q3 = np.max(q3_data) # Calculate the maximum value within Q3

    return max_in_q3

In [6]:
# Creating 'accel' column, to estimate potential maximum movements from one frame to the next
players_df['accel'] = 0 #initialize alternate method of calculating acceleration

for player, group in full_games_df.groupby('nflId'):
    maxAccel = calcQ3Max(group) #as a function of change in speed from frame to frame
    players_df['accel'] = np.where(players_df['nflId'] == player, maxAccel, players_df['accel'])

In [7]:
# Clear all rows after tackle takes place
def mark_rows_to_clear(group):
    max_frame_id = group.loc[group['event'] == 'tackle', 'frameId'].max()
    group['rowsToClear'] = np.where(group['frameId'] <= max_frame_id, 1, 0)
    return group

full_games_df['rowsToClear'] = 0
full_games_df = full_games_df.groupby('unique_key').apply(mark_rows_to_clear)
full_games_df = full_games_df[full_games_df['rowsToClear'] > 0]
full_games_df = full_games_df.drop('rowsToClear', axis=1) 

# Filter out frames where the ball isn't within 1 yard radius of the ball carrier
plays_df = pd.read_csv('plays.csv')
plays_df['unique_key'] = plays_df['gameId'].astype(str) + '_' + plays_df['playId'].astype(str)
full_games_df = pd.merge(full_games_df, plays_df[['unique_key','possessionTeam', 'defensiveTeam', 'playDescription', 'ballCarrierId', 'prePenaltyPlayResult']], on='unique_key', how='left')

def calculate_distance(group):
    ball_carrier_rows = group[group['nflId'] == group['ballCarrierId']]
    football_rows = group[group['club'] == 'football']

    # Select relevant columns early
    ball_carrier_cols = ball_carrier_rows[['frameId', 'unique_key', 'x', 'y']]
    football_cols = football_rows[['frameId', 'unique_key', 'x', 'y']]

    # Merge the ball carrier and football rows based on frameId
    merged_df = pd.merge(ball_carrier_cols, football_cols, on=['frameId', 'unique_key'], suffixes=('_ball_carrier', '_football'))

    # Calculate the distance between positions
    group['dist_2_ball'] = ((merged_df[['x_ball_carrier', 'y_ball_carrier']] - merged_df[['x_football', 'y_football']]) ** 2).sum(axis=1) ** 0.5
    
    return group

# Create an empty column for 'dist_2_ball'
full_games_df['dist_2_ball'] = np.nan
full_games_df = full_games_df.groupby('unique_key').apply(calculate_distance)



In [8]:
# Creating potential distance travelled column which reflects current speed plus calculated ability to accelerate in a single frame.
accel_column = players_df[['nflId', 'accel']] # Extract the relevant column from players_df

full_games_df = pd.merge(full_games_df, accel_column, on='nflId', how='left') # Merge DataFrames based on 'nflId'

full_games_df['time'] = pd.to_datetime(full_games_df['time']) # Convert 'time' column to datetime if it's not already

def calculate_deltas_within_group(group):# Function to calculate frame durations within each group
    group['potDistTravelled'] = group['accel'] + (group['s']/10)
    return group

# Apply the function to each group based on 'unique_key'
full_games_df = full_games_df.groupby('unique_key', group_keys=False).apply(calculate_deltas_within_group)


In [9]:
def calculate_closing_speed(v1, h1, v2, h2):
    # Convert headings to radians
    h1_rad, h2_rad = math.radians(h1), math.radians(h2)

    # Calculate closing speed
    closing_speed = math.sqrt((v2 * math.cos(h2_rad) - v1 * math.cos(h1_rad))**2 +
                              (v2 * math.sin(h2_rad) - v1 * math.sin(h1_rad))**2)

    # Include the sign of the relative velocity
    closing_speed *= math.copysign(1, v2 * math.cos(h2_rad) - v1 * math.cos(h1_rad))

    return closing_speed

In [10]:
def calcAllDistDeltas(group):
    columns = ['unique_key', 'frameId', 'primaryTackler', 'secondaryTackler', 'ballCarrier', 'primaryQuality', 'secondaryQuality']
    new_frames = []

    for i in group['frameId'].unique():
        temp_frame = pd.DataFrame(columns=columns)
        
        currentFrame = group[group['frameId'] == i]

        offenseIds = currentFrame[currentFrame['club'] == currentFrame['possessionTeam']]['nflId'].unique()
        defenseIds = currentFrame[(currentFrame['club'] != currentFrame['possessionTeam']) & (currentFrame['club'] != 'football')]['nflId'].unique()

        distDelta_frame = pd.DataFrame(index=offenseIds, columns=defenseIds)

        x_values = currentFrame.set_index('nflId')['x']
        y_values = currentFrame.set_index('nflId')['y']

        for offenseId in offenseIds:
            for defenseId in defenseIds:
                x_diff = x_values[offenseId] - x_values[defenseId]
                y_diff = y_values[offenseId] - y_values[defenseId]

                distance = np.sqrt(x_diff**2 + y_diff**2)
                distDelta_frame.loc[offenseId, defenseId] = distance

        defRanges = currentFrame.loc[currentFrame['nflId'].isin(defenseIds), ['nflId', 'potDistTravelled']].set_index('nflId')['potDistTravelled']
        defHeadings = currentFrame.loc[currentFrame['nflId'].isin(defenseIds), ['nflId', 'o']].set_index('nflId')['o']
       
        ballCarrierId = currentFrame['ballCarrierId'].head(11)
       
        if currentFrame['playDirection'].iloc[0] == 'left':
            carrierHeading = 270
        else:
            carrierHeading = 90

        carrierProximity = distDelta_frame.loc[distDelta_frame.index == ballCarrierId]
        distDelta_frame = distDelta_frame.drop(ballCarrierId.unique(), axis=0, errors='ignore')

        minDefDist = distDelta_frame.min(axis=0).T
       
        ballCarrierId = currentFrame['ballCarrierId'].head(10)
        carrierSpeed = currentFrame.loc[currentFrame['nflId'].isin(currentFrame['ballCarrierId'].head(1)), ['nflId', 'potDistTravelled']].set_index('nflId')['potDistTravelled']
       
        defClosingSpeeds = pd.DataFrame()
        for j in defenseIds:
            tempDefenderSpeed = defRanges.loc[j]
            tempDefenderHeading = defHeadings.loc[j]
            closingSpeed = calculate_closing_speed(carrierSpeed, carrierHeading, tempDefenderSpeed, tempDefenderHeading)
            defClosingSpeeds.loc[j, 'ClosingSpeed'] = closingSpeed
       
        carrierApproachTime = abs(carrierProximity / defClosingSpeeds['ClosingSpeed'])
       
        possiblePrimarys = np.where(minDefDist < defRanges*0.7878, 100, carrierApproachTime) #tuned to 0.7878 based on multiple trials
        possiblePrimarys = pd.DataFrame(possiblePrimarys, columns=distDelta_frame.columns).T
        possiblePrimarys.sort_values(by=possiblePrimarys.columns[0], ascending=True, inplace=True)

        primaryTackler = int(possiblePrimarys.index[0]) # Selects the defensive player with shortest carrierApproachTime
        secondaryTackler = int(possiblePrimarys.index[1])
        
        # extract carrierApproachTime for primary and secondary tacklers
        primaryQuality = possiblePrimarys.loc[primaryTackler]
        secondaryQuality = possiblePrimarys.loc[secondaryTackler]

        # Append the frame data to the list
        frame_data = [currentFrame['unique_key'].unique()[0], i, primaryTackler, secondaryTackler, ballCarrierId.unique()[0], primaryQuality, secondaryQuality]
        new_frames.append(frame_data)

    # Create a new DataFrame from the list of frames
    new_group = pd.DataFrame(new_frames, columns=columns)

    return new_group


In [11]:
final_frames = []
totalSkipped = 0
unique_keys = full_games_df['unique_key'].unique()

for i, key in enumerate(unique_keys, start=1):
    try:
        subset_df = full_games_df[full_games_df['unique_key'] == key]
        subset_df = calcAllDistDeltas(subset_df)
        final_frames.append(subset_df)
    except Exception as e:
        print(f"Skipped processing key: {key}. Error: {e}")
        totalSkipped += 1

    print(f"Processed: {i} of {len(unique_keys)}")

print("Total skipped: " + str(totalSkipped))

final_dataset = pd.concat(final_frames, ignore_index=True)
final_dataset.to_csv('primaryAndSecondary.csv', index=False)

Processed: 1 of 10044
Processed: 2 of 10044
Processed: 3 of 10044
Processed: 4 of 10044
Processed: 5 of 10044
Processed: 6 of 10044
Processed: 7 of 10044
Processed: 8 of 10044
Processed: 9 of 10044
Processed: 10 of 10044
Processed: 11 of 10044
Processed: 12 of 10044
Processed: 13 of 10044
Processed: 14 of 10044
Processed: 15 of 10044
Processed: 16 of 10044
Processed: 17 of 10044
Processed: 18 of 10044
Processed: 19 of 10044
Processed: 20 of 10044
Processed: 21 of 10044
Processed: 22 of 10044
Processed: 23 of 10044
Processed: 24 of 10044
Processed: 25 of 10044
Processed: 26 of 10044
Processed: 27 of 10044
Processed: 28 of 10044
Processed: 29 of 10044
Processed: 30 of 10044
Processed: 31 of 10044
Processed: 32 of 10044
Processed: 33 of 10044
Processed: 34 of 10044
Processed: 35 of 10044
Processed: 36 of 10044
Processed: 37 of 10044
Processed: 38 of 10044
Processed: 39 of 10044
Processed: 40 of 10044
Processed: 41 of 10044
Processed: 42 of 10044
Processed: 43 of 10044
Processed: 44 of 100

In [12]:
# change data frame name and clear up memory
full_games_df = final_dataset
final_dataset = pd.DataFrame()
tackles_df = pd.read_csv('tackles.csv')

grouped_df = full_games_df.groupby('unique_key', as_index=False)

# Calculate primaryTackler and secondaryTackler DataFrames using agg
agg_df = grouped_df.agg({'primaryTackler': lambda x: x.value_counts().idxmax(),
                         'secondaryTackler': lambda x: x.value_counts().idxmax()})

# Rename the columns
agg_df.rename(columns={'primaryTackler': 'primaryTackler', 'secondaryTackler': 'secondaryTackler'}, inplace=True)

# Merge the DataFrames based on 'unique_key'
tacklers_by_play_df = pd.merge(agg_df[['unique_key', 'primaryTackler']], agg_df[['unique_key', 'secondaryTackler']], on='unique_key')

# Rename the columns to match the desired output
tacklers_by_play_df.columns = ['unique_key', 'primaryTackler', 'secondaryTackler']


In [13]:
# Find the quality score of each primary tackler
columns = ['ID', 'Primary Opportunity Score (inverse)']
primeQuality = pd.DataFrame(columns=columns)

for playerId in full_games_df['primaryTackler'].unique():
    relevantFrames = full_games_df[full_games_df['primaryTackler'] == playerId]
    avgQuality = relevantFrames['primaryQuality'].mean()

    # Append data to the DataFrame using a dictionary
    primeQuality = primeQuality.append({'ID': playerId, 'Primary Opportunity Score (inverse)': avgQuality}, ignore_index=True)

# Find the quality score of each tackler
columns = ['ID', 'Secondary Opportunity Score (inverse)']
secondQuality = pd.DataFrame(columns=columns)

for playerId in full_games_df['secondaryTackler'].unique():
    relevantFrames = full_games_df[full_games_df['secondaryTackler'] == playerId]
    avgQuality = relevantFrames['secondaryQuality'].mean()

    # Append data to the DataFrame using a dictionary
    secondQuality = secondQuality.append({'ID': playerId, 'Secondary Opportunity Score (inverse)': avgQuality}, ignore_index=True)

    

In [14]:

total_plays = len(tackles_df)

full_games_df = tacklers_by_play_df
# Create a new column 'unique_key' in tackles_df by combining 'gameId' + "_" + 'playId'
tackles_df['unique_key'] = tackles_df['gameId'].astype(str) + "_" + tackles_df['playId'].astype(str)

# Rename 'nflId' column in tackles_df to 'actualTackler'
tackles_df.rename(columns={'nflId': 'actualTackler'}, inplace=True)

# Join tackles_df['actualTackler'] to full_games_df on 'unique_key'
full_games_df = full_games_df.merge(tackles_df[['unique_key', 'actualTackler', 'tackle']], how='left', on='unique_key')

# Drop rows where 'tackle' is not equal to 1
full_games_df = full_games_df[full_games_df['tackle'] == 1]

tackler_ratios_df = pd.DataFrame()

# Create a list of all unique values in 'primaryTackler', 'secondaryTackler', and 'actualTackler'
unique_ids = list(set(full_games_df['primaryTackler'].unique()) |
                     set(full_games_df['secondaryTackler'].unique()) |
                     set(full_games_df['actualTackler'].unique()))

for i in unique_ids:
    try:
        subset1 = full_games_df[(full_games_df['primaryTackler'] == i) & (full_games_df['actualTackler'] == i)]
        success_as_prime = len(subset1)
        
        subset2 = full_games_df[(full_games_df['secondaryTackler'] == i) & (full_games_df['actualTackler'] == i)]
        success_as_second = len(subset2)
        
        subset3 = full_games_df[full_games_df['primaryTackler'] == i]
        count_as_prime = len(subset3)
        
        subset4 = full_games_df[full_games_df['secondaryTackler'] == i]
        count_as_second = len(subset4)

        subset5 = full_games_df[full_games_df['actualTackler'] == i]
        player_tackles = len(subset5)

        prime_contributionRate = success_as_prime/total_plays
        sec_contributionRate = success_as_second/total_plays

        prime_success_ratio = success_as_prime / count_as_prime
        second_success_ratio = success_as_second / count_as_second
        prime_ratio = prime_success_ratio * prime_contributionRate * 1000
        second_ratio = second_success_ratio * sec_contributionRate * 1000
        tackler_ratios_df = tackler_ratios_df.append({'ID': i, 'Ace Success Ratio': prime_success_ratio, 'Wingman Success Ratio': second_success_ratio, 'Ace Contribution Score': prime_ratio, 'Wingman Contribution Score': second_ratio, 'Ace Tackles': success_as_prime, 'Wingman Tackles': success_as_second, 'Total Tackles':player_tackles}, ignore_index=True)
    except:
        continue
        
# Sort the DataFrame by 'Ace Score' column in descending order
tackler_ratios_df = tackler_ratios_df.sort_values(by=['Ace Contribution Score', 'Wingman Contribution Score'], ascending=[False, False])

tackler_ratios_df = pd.merge(tackler_ratios_df, primeQuality, how='left', on='ID') # Add quality score
tackler_ratios_df = pd.merge(tackler_ratios_df, secondQuality, how='left', on='ID') # Add quality score
###### ACE Rating ######
# Normalize Contribution Score
min_contribution = tackler_ratios_df['Ace Contribution Score'].min()
max_contribution = tackler_ratios_df['Ace Contribution Score'].max()
tackler_ratios_df['Normalized Primary Contribution Score'] = (tackler_ratios_df['Ace Contribution Score'] - min_contribution) / (max_contribution - min_contribution)

# Normalize Opportunity Score
min_opportunity = tackler_ratios_df['Primary Opportunity Score (inverse)'].min()
max_opportunity = tackler_ratios_df['Primary Opportunity Score (inverse)'].max()
tackler_ratios_df['Normalized Primary Opportunity Score'] = (tackler_ratios_df['Primary Opportunity Score (inverse)'] - min_opportunity) / (max_opportunity - min_opportunity)

# Calculate Rating
tackler_ratios_df['Ace Rating'] = tackler_ratios_df['Normalized Primary Contribution Score'] * tackler_ratios_df['Normalized Primary Opportunity Score']

##### Wingman Rating ########
# Normalize Contribution Score
min_contribution = tackler_ratios_df['Wingman Contribution Score'].min()
max_contribution = tackler_ratios_df['Wingman Contribution Score'].max()
tackler_ratios_df['Normalized Wingman Contribution Score'] = (tackler_ratios_df['Wingman Contribution Score'] - min_contribution) / (max_contribution - min_contribution)

# Normalize Opportunity Score
min_opportunity = tackler_ratios_df['Secondary Opportunity Score (inverse)'].min()
max_opportunity = tackler_ratios_df['Secondary Opportunity Score (inverse)'].max()
tackler_ratios_df['Normalized Wingman Opportunity Score'] = (tackler_ratios_df['Secondary Opportunity Score (inverse)'] - min_opportunity) / (max_opportunity - min_opportunity)

# Calculate Rating
tackler_ratios_df['Wingman Rating'] = tackler_ratios_df['Normalized Wingman Contribution Score'] * tackler_ratios_df['Normalized Wingman Opportunity Score']

# Set option to display all rows
pd.set_option('display.max_rows', None)
tackler_ratios_df.to_csv('finalResult.csv', index=False) #previously 'qualityresultX.csv' for tuning
# Print the DataFrame with ratios
print("\nTackler Ratios DataFrame (Descending Order by Ace Score):")
print(tackler_ratios_df)



Tackler Ratios DataFrame (Descending Order by Ace Score):
          ID  Ace Success Ratio  Wingman Success Ratio  \
0    46269.0           0.875000               0.458333   
1    53489.0           0.928571               0.210526   
2    43353.0           0.880000               0.142857   
3    43325.0           0.821429               0.052632   
4    47913.0           0.950000               0.105263   
5    48516.0           0.741935               0.307692   
6    54500.0           0.807692               0.222222   
7    41243.0           0.675676               0.235294   
8    53487.0           0.758621               0.250000   
9    42427.0           0.900000               0.222222   
10   38588.0           0.944444               0.291667   
11   38577.0           0.944444               0.230769   
12   46085.0           0.800000               0.263158   
13   52527.0           0.666667               0.192308   
14   46304.0           0.750000               0.080000   
15   52435.0 

In [15]:
tackler_subset = tackler_ratios_df.head(10)
player_info = pd.read_csv('players.csv')

merged_df = tackler_subset.merge(player_info, left_on='ID', right_on='nflId', how='left')

# Print the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df)


Merged DataFrame:
        ID  Ace Success Ratio  Wingman Success Ratio  Ace Contribution Score  \
0  46269.0           0.875000               0.458333                1.405945   
1  53489.0           0.928571               0.210526                1.385450   
2  43353.0           0.880000               0.142857                1.110984   
3  43325.0           0.821429               0.052632                1.084176   
4  47913.0           0.950000               0.105263                1.035809   
5  48516.0           0.741935               0.307692                0.979256   
6  54500.0           0.807692               0.222222                0.973347   
7  41243.0           0.675676               0.235294                0.969350   
8  53487.0           0.758621               0.250000                0.957744   
9  42427.0           0.900000               0.222222                0.929645   

   Wingman Contribution Score  Ace Tackles  Wingman Tackles  Total Tackles  \
0                    0

In [16]:
elapsed_time = time.time() - start_time
print(f"Total run time: {elapsed_time} seconds")

Total run time: 10789.271682024002 seconds


In [17]:
print(len(full_games_df))
# Calculate the sum of 'Ace Tackles' + 'Wingman Tackles'
sum_ace_wingman_tackles = sum(tackler_ratios_df['Ace Tackles'] + tackler_ratios_df['Wingman Tackles'])
print(f"Sum of Ace Tackles + Wingman Tackles: {sum_ace_wingman_tackles}")

# Calculate the sum of 'Total Tackles'
sum_total_tackles = sum(tackler_ratios_df['Total Tackles'])
print(f"Sum of Total Tackles: {sum_total_tackles}")

8115
Sum of Ace Tackles + Wingman Tackles: 5302.0
Sum of Total Tackles: 8003.0
